In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score
import warnings
warnings.filterwarnings('ignore')

class ImprovedCreditScoring:
    def __init__(self, n_splits=5, random_state=42):
        self.n_splits = n_splits
        self.random_state = random_state

        # Modified base models with better hyperparameters
        self.base_models = {
            'xgb1': xgb.XGBClassifier(
                n_estimators=300,
                learning_rate=0.03,
                max_depth=5,
                subsample=0.85,
                colsample_bytree=0.8,
                min_child_weight=3,
                scale_pos_weight=2,  # Handle class imbalance
                random_state=random_state
            ),
            'xgb2': xgb.XGBClassifier(
                n_estimators=300,
                learning_rate=0.03,
                max_depth=4,
                subsample=0.8,
                colsample_bytree=0.9,
                min_child_weight=4,
                scale_pos_weight=2,
                random_state=random_state
            ),
            'rf': RandomForestClassifier(
                n_estimators=300,
                max_depth=6,
                min_samples_split=30,
                min_samples_leaf=15,
                class_weight='balanced',  # Handle class imbalance
                random_state=random_state
            )
        }

        # Use L2 regularization in meta model
        self.meta_model = LogisticRegression(C=0.8, class_weight='balanced', random_state=random_state)
        self.scaler = RobustScaler()  # More robust to outliers

    def preprocess_data(self, data):
        """Enhanced preprocessing with feature engineering"""
        # Handle missing values more sophisticatedly
        data = data.copy()

        # Separate features by type
        onus_cols = [col for col in data.columns if col.startswith('onus_attribute')]
        trans_cols = [col for col in data.columns if col.startswith('transaction_attribute')]
        bureau_cols = [col for col in data.columns if col.startswith('bureau') and not col.startswith('bureau_enquiry')]
        enquiry_cols = [col for col in data.columns if col.startswith('bureau_enquiry')]

        # Fill missing values based on feature type
        data[onus_cols] = data[onus_cols].fillna(data[onus_cols].median())
        data[trans_cols] = data[trans_cols].fillna(0)  # Assume missing transactions are 0
        data[bureau_cols] = data[bureau_cols].fillna(data[bureau_cols].median())
        data[enquiry_cols] = data[enquiry_cols].fillna(0)  # Assume missing enquiries are 0

        # Feature engineering
        # Transaction ratios
        for col in trans_cols:
            if 'value' in col:
                corresponding_count = col.replace('value', 'count')
                if corresponding_count in trans_cols:
                    data[f'{col}_per_transaction'] = data[col] / (data[corresponding_count] + 1)

        # Bureau ratios
        total_accounts = data[[col for col in bureau_cols if 'count' in col]].sum(axis=1)
        for col in bureau_cols:
            if 'count' in col:
                data[f'{col}_ratio'] = data[col] / (total_accounts + 1)

        # Scale features
        numerical_cols = onus_cols + trans_cols + bureau_cols + enquiry_cols
        data[numerical_cols] = self.scaler.fit_transform(data[numerical_cols])

        return data

    def generate_meta_features(self, X, y=None, is_train=True):
        """Generate meta-features using stratified k-fold"""
        if is_train:
            meta_features = np.zeros((X.shape[0], len(self.base_models)))
            # Use StratifiedKFold for better handling of imbalanced data
            kf = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=self.random_state)

            for i, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
                X_train_fold = X.iloc[train_idx]
                y_train_fold = y.iloc[train_idx]
                X_valid_fold = X.iloc[valid_idx]

                for j, (name, model) in enumerate(self.base_models.items()):
                    model.fit(X_train_fold, y_train_fold)
                    meta_features[valid_idx, j] = model.predict_proba(X_valid_fold)[:, 1]

            return meta_features
        else:
            meta_features = np.zeros((X.shape[0], len(self.base_models)))
            for j, (name, model) in enumerate(self.base_models.items()):
                meta_features[:, j] = model.predict_proba(X)[:, 1]
            return meta_features

    def fit(self, X, y):
        """Enhanced fitting with feature importance analysis"""
        # Generate meta-features
        meta_features = self.generate_meta_features(X, y, is_train=True)

        # Fit meta-model
        self.meta_model.fit(meta_features, y)

        # Store feature importances from XGBoost models
        self.feature_importances_ = {}
        for name, model in self.base_models.items():
            if isinstance(model, xgb.XGBClassifier):
                self.feature_importances_[name] = dict(zip(X.columns, model.feature_importances_))

        # Retrain base models on full data
        for name, model in self.base_models.items():
            model.fit(X, y)

        return self

    def predict_proba(self, X):
        """Generate probability predictions with optional calibration"""
        meta_features = self.generate_meta_features(X, is_train=False)
        return self.meta_model.predict_proba(meta_features)[:, 1]

    def get_top_features(self, n=20):
        """Get top n most important features from XGBoost models"""
        all_importances = {}
        for name, importances in self.feature_importances_.items():
            for feature, importance in importances.items():
                if feature not in all_importances:
                    all_importances[feature] = 0
                all_importances[feature] += importance

        return dict(sorted(all_importances.items(), key=lambda x: x[1], reverse=True)[:n])

In [8]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score, accuracy_score, confusion_matrix, classification_report
import warnings
import zipfile
import os
warnings.filterwarnings('ignore')

class CreditScorer:
    def __init__(self, n_splits=5, random_state=42):
        self.n_splits = n_splits
        self.random_state = random_state

        # Define base models
        self.base_models = {
            'xgb1': xgb.XGBClassifier(
                n_estimators=300,
                learning_rate=0.03,
                max_depth=5,
                subsample=0.85,
                colsample_bytree=0.8,
                min_child_weight=3,
                scale_pos_weight=2,
                random_state=random_state
            ),
            'xgb2': xgb.XGBClassifier(
                n_estimators=300,
                learning_rate=0.03,
                max_depth=4,
                subsample=0.8,
                colsample_bytree=0.9,
                min_child_weight=4,
                scale_pos_weight=2,
                random_state=random_state
            ),
            'rf': RandomForestClassifier(
                n_estimators=300,
                max_depth=6,
                min_samples_split=30,
                min_samples_leaf=15,
                class_weight='balanced',
                random_state=random_state
            )
        }
        self.meta_model = LogisticRegression(C=0.8, class_weight='balanced', random_state=random_state)

    def generate_meta_features(self, X, y=None, is_train=True):
        if is_train:
            meta_features = np.zeros((X.shape[0], len(self.base_models)))
            kf = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=self.random_state)

            for i, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
                X_train_fold = X.iloc[train_idx]
                y_train_fold = y.iloc[train_idx]
                X_valid_fold = X.iloc[valid_idx]

                for j, (name, model) in enumerate(self.base_models.items()):
                    model.fit(X_train_fold, y_train_fold)
                    meta_features[valid_idx, j] = model.predict_proba(X_valid_fold)[:, 1]

            return meta_features
        else:
            meta_features = np.zeros((X.shape[0], len(self.base_models)))
            for j, (name, model) in enumerate(self.base_models.items()):
                meta_features[:, j] = model.predict_proba(X)[:, 1]
            return meta_features

    def fit(self, X, y):
        meta_features = self.generate_meta_features(X, y, is_train=True)
        self.meta_model.fit(meta_features, y)

        # Retrain base models on full data
        for name, model in self.base_models.items():
            model.fit(X, y)

        return self

    def predict_proba(self, X):
        meta_features = self.generate_meta_features(X, is_train=False)
        return self.meta_model.predict_proba(meta_features)[:, 1]

def extract_zip(zip_path, extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    extracted_files = [f for f in os.listdir(extract_path) if f.endswith('.csv')]
    if not extracted_files:
        raise ValueError("No CSV file found in the extracted contents")
    return os.path.join(extract_path, extracted_files[0])

def print_metrics(metrics):
    print("\nModel Performance Metrics:")
    print("=" * 50)
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"AUC Score: {metrics['auc_score']:.4f}")
    print(f"Average Precision: {metrics['avg_precision']:.4f}")

    print("\nConfusion Matrix:")
    print("-" * 50)
    print(f"True Negatives: {metrics['confusion_matrix']['tn']}")
    print(f"False Positives: {metrics['confusion_matrix']['fp']}")
    print(f"False Negatives: {metrics['confusion_matrix']['fn']}")
    print(f"True Positives: {metrics['confusion_matrix']['tp']}")


print("Loading development data...")
dev_data = pd.read_csv('/content/cleaned_dev.csv')

# Create temporary directory for validation data
temp_dir = 'temp_extracted'
os.makedirs(temp_dir, exist_ok=True)

try:
    # Extract and load validation data
    print("Loading validation data...")
    val_csv_path = extract_zip('/content/validation_data_to_be_shared 3.zip', temp_dir)
    val_data = pd.read_csv(val_csv_path)

    # Prepare features and target
    target = 'bad_flag'
    features = [col for col in dev_data.columns if col not in [target, 'account_number']]

    print("Training model...")
    # Initialize and train model
    model = CreditScorer()
    model.fit(dev_data[features], dev_data[target])

    print("Generating predictions...")
    # Generate predictions
    val_predictions = model.predict_proba(val_data[features])

    # Create submission file
    submission = pd.DataFrame({
        'account_number': val_data['account_number'],
        'predicted_probability': val_predictions
    })

    print("Saving predictions...")
    # Save predictions
    submission.to_csv('predictions1.csv', index=False)

    print("Done! Predictions saved to 'predictions.csv'")

finally:
    # Clean up temporary files
    import shutil
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)

Loading development data...
Loading validation data...
Training model...
Generating predictions...
Saving predictions...
Done! Predictions saved to 'predictions.csv'


In [9]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score, accuracy_score, confusion_matrix, classification_report
import warnings
import zipfile
import os
warnings.filterwarnings('ignore')

class CreditScorer:
    def __init__(self, n_splits=5, random_state=42):
        self.n_splits = n_splits
        self.random_state = random_state

        # Define base models
        self.base_models = {
            'xgb1': xgb.XGBClassifier(
                n_estimators=300,
                learning_rate=0.03,
                max_depth=5,
                subsample=0.85,
                colsample_bytree=0.8,
                min_child_weight=3,
                scale_pos_weight=2,
                random_state=random_state
            ),
            'xgb2': xgb.XGBClassifier(
                n_estimators=300,
                learning_rate=0.03,
                max_depth=4,
                subsample=0.8,
                colsample_bytree=0.9,
                min_child_weight=4,
                scale_pos_weight=2,
                random_state=random_state
            ),
            'rf': RandomForestClassifier(
                n_estimators=300,
                max_depth=6,
                min_samples_split=30,
                min_samples_leaf=15,
                class_weight='balanced',
                random_state=random_state
            )
        }
        self.meta_model = LogisticRegression(C=0.8, class_weight='balanced', random_state=random_state)

    def generate_meta_features(self, X, y=None, is_train=True):
        if is_train:
            meta_features = np.zeros((X.shape[0], len(self.base_models)))
            kf = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=self.random_state)

            # Store CV scores
            self.cv_scores = []

            for i, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
                X_train_fold = X.iloc[train_idx]
                y_train_fold = y.iloc[train_idx]
                X_valid_fold = X.iloc[valid_idx]
                y_valid_fold = y.iloc[valid_idx]

                fold_preds = np.zeros(len(valid_idx))
                for j, (name, model) in enumerate(self.base_models.items()):
                    model.fit(X_train_fold, y_train_fold)
                    proba = model.predict_proba(X_valid_fold)[:, 1]
                    meta_features[valid_idx, j] = proba
                    fold_preds += proba / len(self.base_models)

                # Calculate and store fold score
                fold_score = roc_auc_score(y_valid_fold, fold_preds)
                self.cv_scores.append(fold_score)
                print(f"Fold {i+1} AUC: {fold_score:.4f}")

            print(f"\nMean CV AUC: {np.mean(self.cv_scores):.4f} (+/- {np.std(self.cv_scores):.4f})")
            return meta_features
        else:
            meta_features = np.zeros((X.shape[0], len(self.base_models)))
            for j, (name, model) in enumerate(self.base_models.items()):
                meta_features[:, j] = model.predict_proba(X)[:, 1]
            return meta_features

    def fit(self, X, y):
        print("Generating meta-features and CV scores...")
        meta_features = self.generate_meta_features(X, y, is_train=True)
        self.meta_model.fit(meta_features, y)

        print("\nTraining base models on full data...")
        for name, model in self.base_models.items():
            model.fit(X, y)

        return self

    def predict_proba(self, X):
        meta_features = self.generate_meta_features(X, is_train=False)
        return self.meta_model.predict_proba(meta_features)[:, 1]

    def evaluate(self, X, y, threshold=0.5):
        """Evaluate model performance with multiple metrics"""
        y_pred_proba = self.predict_proba(X)
        y_pred = (y_pred_proba >= threshold).astype(int)

        tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()

        metrics = {
            'accuracy': accuracy_score(y, y_pred),
            'auc_score': roc_auc_score(y, y_pred_proba),
            'avg_precision': average_precision_score(y, y_pred_proba),
            'confusion_matrix': {
                'tn': tn,
                'fp': fp,
                'fn': fn,
                'tp': tp
            }
        }

        print("\nModel Performance Metrics:")
        print("=" * 50)
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print(f"AUC Score: {metrics['auc_score']:.4f}")
        print(f"Average Precision: {metrics['avg_precision']:.4f}")

        print("\nConfusion Matrix:")
        print("-" * 50)
        print(f"True Negatives: {tn}")
        print(f"False Positives: {fp}")
        print(f"False Negatives: {fn}")
        print(f"True Positives: {tp}")

        # Calculate and print additional metrics
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        print("\nAdditional Metrics:")
        print("-" * 50)
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")

        return metrics


# Load development data
print("Loading development data...")
dev_data = pd.read_csv('/content/cleaned_dev.csv')

# Prepare features and target
target = 'bad_flag'
features = [col for col in dev_data.columns if col not in [target, 'account_number']]

# Split development data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    dev_data[features],
    dev_data[target],
    test_size=0.2,
    random_state=42,
    stratify=dev_data[target]  # Ensure balanced split
)

print(f"\nData split:")
print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")
print(f"Positive class distribution in train: {(y_train==1).mean():.4f}")
print(f"Positive class distribution in test: {(y_test==1).mean():.4f}")

# Initialize and train model
print("\nTraining model...")
model = CreditScorer()
model.fit(X_train, y_train)

# Evaluate on test set
print("\nEvaluating on test set...")
test_metrics = model.evaluate(X_test, y_test)

# Create temporary directory for validation data
temp_dir = 'temp_extracted'
os.makedirs(temp_dir, exist_ok=True)

try:
    # Extract and load validation data
    print("\nLoading validation data...")
    val_csv_path = extract_zip('/content/validation_data_to_be_shared 3.zip', temp_dir)
    val_data = pd.read_csv(val_csv_path)

    print("\nGenerating predictions for validation data...")
    # Now retrain on full development data before making validation predictions
    print("\nRetraining model on full development data...")
    model = CreditScorer()
    model.fit(dev_data[features], dev_data[target])

    # Generate predictions for validation data
    val_predictions = model.predict_proba(val_data[features])

    # Create submission file
    submission = pd.DataFrame({
        'account_number': val_data['account_number'],
        'predicted_probability': val_predictions
    })

    print("\nSaving predictions...")
    submission.to_csv('predictions2.csv', index=False)
    print("\nDone! Predictions saved to 'predictions.csv'")

finally:
    # Clean up temporary files
    import shutil
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)

Loading development data...

Data split:
Training samples: 2376
Testing samples: 595
Positive class distribution in train: 0.1431
Positive class distribution in test: 0.1429

Training model...
Generating meta-features and CV scores...
Fold 1 AUC: 0.7091
Fold 2 AUC: 0.7339
Fold 3 AUC: 0.7375
Fold 4 AUC: 0.7425
Fold 5 AUC: 0.7314

Mean CV AUC: 0.7309 (+/- 0.0115)

Training base models on full data...

Evaluating on test set...

Model Performance Metrics:
Accuracy: 0.7899
AUC Score: 0.9189
Average Precision: 0.5699

Confusion Matrix:
--------------------------------------------------
True Negatives: 390
False Positives: 120
False Negatives: 5
True Positives: 80

Additional Metrics:
--------------------------------------------------
Precision: 0.4000
Recall: 0.9412
F1 Score: 0.5614

Loading validation data...

Generating predictions for validation data...

Retraining model on full development data...
Generating meta-features and CV scores...
Fold 1 AUC: 0.7370
Fold 2 AUC: 0.7145
Fold 3 AUC

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import warnings
import zipfile
import os
warnings.filterwarnings('ignore')

class ImprovedCreditScorer:
    def __init__(self, n_splits=5, random_state=42):
        self.n_splits = n_splits
        self.random_state = random_state
        self.scaler = StandardScaler()
        self.feature_names_ = None

        # Define base models with adjusted parameters
        self.base_models = {
            'xgb1': xgb.XGBClassifier(
                n_estimators=500,
                learning_rate=0.02,
                max_depth=4,
                subsample=0.8,
                colsample_bytree=0.8,
                min_child_weight=4,
                scale_pos_weight=3,  # Increased penalty for false positives
                random_state=random_state
            ),
            'xgb2': xgb.XGBClassifier(
                n_estimators=500,
                learning_rate=0.02,
                max_depth=3,
                subsample=0.85,
                colsample_bytree=0.9,
                min_child_weight=5,
                scale_pos_weight=3,
                random_state=random_state
            ),
            'rf': RandomForestClassifier(
                n_estimators=500,
                max_depth=5,
                min_samples_split=25,
                min_samples_leaf=10,
                class_weight={0: 1, 1: 3},  # Increased penalty for false positives
                random_state=random_state
            )
        }
        self.meta_model = LogisticRegression(C=0.6, class_weight={0: 1, 1: 3}, random_state=random_state)

        # Initialize sampling strategy
        self.sampler = Pipeline([
            ('under', RandomUnderSampler(sampling_strategy=0.5, random_state=random_state)),
            ('over', SMOTE(sampling_strategy=0.8, random_state=random_state))
        ])

    def engineer_features(self, X):
        """Engineer features from transaction, bureau, and onus attributes"""
        X_new = X.copy()

        # Transaction attribute aggregations
        tx_cols = [col for col in X.columns if col.startswith('transaction_attribute_')]
        if tx_cols:
            # Average of non-zero transaction values
            X_new['avg_nonzero_tx'] = X[tx_cols].replace(0, np.nan).mean(axis=1)
            # Count of non-zero transactions
            X_new['nonzero_tx_count'] = (X[tx_cols] != 0).sum(axis=1)
            # Transaction value volatility
            X_new['tx_volatility'] = X[tx_cols].std(axis=1)

        # Bureau data aggregations
        bureau_cols = [col for col in X.columns if col.startswith('bureau_')]
        if bureau_cols:
            # Average bureau scores
            X_new['avg_bureau_score'] = X[bureau_cols].mean(axis=1)
            # Standard deviation of bureau scores
            X_new['bureau_score_spread'] = X[bureau_cols].std(axis=1)
            # Count of non-zero bureau entries
            X_new['active_bureau_count'] = (X[bureau_cols] != 0).sum(axis=1)

        # Onus attribute aggregations
        onus_cols = [col for col in X.columns if col.startswith('onus_attribute_')]
        if onus_cols:
            # Average of onus attributes
            X_new['avg_onus_score'] = X[onus_cols].mean(axis=1)
            # Volatility in onus attributes
            X_new['onus_volatility'] = X[onus_cols].std(axis=1)

        # Bureau enquiry features
        enquiry_cols = [col for col in X.columns if col.startswith('bureau_enquiry_')]
        if enquiry_cols:
            # Total number of enquiries
            X_new['total_enquiries'] = X[enquiry_cols].sum(axis=1)
            # Recent enquiry intensity (assuming more recent enquiries are later in the sequence)
            recent_enquiry_cols = enquiry_cols[-10:]  # Last 10 enquiries
            X_new['recent_enquiry_intensity'] = X[recent_enquiry_cols].sum(axis=1)

        # Interaction features
        if 'avg_bureau_score' in X_new.columns and 'total_enquiries' in X_new.columns:
            X_new['bureau_enquiry_risk'] = X_new['avg_bureau_score'] / (X_new['total_enquiries'] + 1)

        if 'nonzero_tx_count' in X_new.columns and 'active_bureau_count' in X_new.columns:
            X_new['activity_ratio'] = X_new['nonzero_tx_count'] / (X_new['active_bureau_count'] + 1)

        # Risk indicators
        if 'avg_bureau_score' in X_new.columns:
            X_new['high_bureau_risk'] = (X_new['avg_bureau_score'] < X_new['avg_bureau_score'].mean()).astype(int)

        if 'total_enquiries' in X_new.columns:
            X_new['high_enquiry_risk'] = (X_new['total_enquiries'] > X_new['total_enquiries'].mean()).astype(int)

        # Drop any features that ended up with NaN values
        X_new = X_new.fillna(0)
        self.feature_names_ = list(X_new.columns)

        return X_new

    def find_optimal_threshold(self, y_true, y_pred_proba):
        """Find optimal classification threshold based on precision-recall curve"""
        precisions, recalls, thresholds = precision_recall_curve(y_true, y_pred_proba)
        f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-10)
        optimal_threshold = thresholds[np.argmax(f1_scores[:-1])]
        return optimal_threshold

    def generate_meta_features(self, X, y=None, is_train=True):
        if is_train:
            meta_features = np.zeros((X.shape[0], len(self.base_models)))
            kf = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=self.random_state)

            self.cv_scores = []
            self.feature_importances_ = {}

            for i, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
                X_train_fold = X.iloc[train_idx]
                y_train_fold = y.iloc[train_idx]
                X_valid_fold = X.iloc[valid_idx]
                y_valid_fold = y.iloc[valid_idx]

                # Apply sampling only to training fold
                X_train_resampled, y_train_resampled = self.sampler.fit_resample(X_train_fold, y_train_fold)

                fold_preds = np.zeros(len(valid_idx))
                for j, (name, model) in enumerate(self.base_models.items()):
                    model.fit(X_train_resampled, y_train_resampled)
                    proba = model.predict_proba(X_valid_fold)[:, 1]
                    meta_features[valid_idx, j] = proba
                    fold_preds += proba / len(self.base_models)

                    # Store feature importances for tree-based models
                    if hasattr(model, 'feature_importances_'):
                        if name not in self.feature_importances_:
                            self.feature_importances_[name] = np.zeros(X.shape[1])
                        self.feature_importances_[name] += model.feature_importances_ / self.n_splits

                fold_score = roc_auc_score(y_valid_fold, fold_preds)
                self.cv_scores.append(fold_score)
                print(f"Fold {i+1} AUC: {fold_score:.4f}")

            print(f"\nMean CV AUC: {np.mean(self.cv_scores):.4f} (+/- {np.std(self.cv_scores):.4f})")
            return meta_features
        else:
            meta_features = np.zeros((X.shape[0], len(self.base_models)))
            for j, (name, model) in enumerate(self.base_models.items()):
                meta_features[:, j] = model.predict_proba(X)[:, 1]
            return meta_features

    def fit(self, X, y):
        print("Engineering features...")
        X_engineered = self.engineer_features(X)

        print("Scaling features...")
        X_scaled = pd.DataFrame(
            self.scaler.fit_transform(X_engineered),
            columns=self.feature_names_,
            index=X_engineered.index
        )

        print("Generating meta-features and CV scores...")
        meta_features = self.generate_meta_features(X_scaled, y, is_train=True)

        self.training_features_ = self.feature_names_

        # Resample data for meta-model
        meta_features_resampled, y_resampled = self.sampler.fit_resample(meta_features, y)
        self.meta_model.fit(meta_features_resampled, y_resampled)

        print("\nTraining base models on full data...")
        X_full_resampled, y_full_resampled = self.sampler.fit_resample(X_scaled, y)
        for name, model in self.base_models.items():
            model.fit(X_full_resampled, y_full_resampled)

        return self

    def predict_proba(self, X):
        X_engineered = self.engineer_features(X)

        # Ensure features match training order
        X_engineered = X_engineered[self.training_features_]

        X_scaled = pd.DataFrame(
            self.scaler.transform(X_engineered),
            columns=self.training_features_,
            index=X_engineered.index
        )
        meta_features = self.generate_meta_features(X_scaled, is_train=False)
        return self.meta_model.predict_proba(meta_features)[:, 1]

    def evaluate(self, X, y, threshold=None):
        """Evalaluate model performance with multiple metrics and optimal threshold"""
        # Ensure features match training features
        X_engineered = self.engineer_features(X)
        X_eval = X_engineered[self.training_features_]

        # Generate predictions
        y_pred_proba = self.predict_proba(X_eval)

        # Find optimal threshold if not provided
        if threshold is None:
            threshold = self.find_optimal_threshold(y, y_pred_proba)

        y_pred = (y_pred_proba >= threshold).astype(int)

        tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()

        metrics = {
            'threshold': threshold,
            'accuracy': accuracy_score(y, y_pred),
            'auc_score': roc_auc_score(y, y_pred_proba),
            'avg_precision': average_precision_score(y, y_pred_proba),
            'confusion_matrix': {
                'tn': tn,
                'fp': fp,
                'fn': fn,
                'tp': tp
            }
        }

        print("\nModel Performance Metrics:")
        print("=" * 50)
        print(f"Optimal Threshold: {threshold:.4f}")
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print(f"AUC Score: {metrics['auc_score']:.4f}")
        print(f"Average Precision: {metrics['avg_precision']:.4f}")

        print("\nConfusion Matrix:")
        print("-" * 50)
        print(f"True Negatives: {tn}")
        print(f"False Positives: {fp}")
        print(f"False Negatives: {fn}")
        print(f"True Positives: {tp}")

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        print("\nAdditional Metrics:")
        print("-" * 50)
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")

        if hasattr(self, 'feature_importances_'):
            print("\nTop 10 Important Features:")
            print("-" * 50)
            for name, importances in self.feature_importances_.items():
                feat_imp = pd.Series(importances, index=self.training_features_)
                print(f"\n{name}:")
                print(feat_imp.nlargest(10).to_string())

        return metrics

# Example usage
if __name__ == "__main__":
    # Load development data
    print("Loading development data...")
    dev_data = pd.read_csv('/content/cleaned_dev.csv')

    # Prepare features and target
    target = 'bad_flag'
    features = [col for col in dev_data.columns if col not in [target, 'account_number']]

    # Split development data
    X_train, X_test, y_train, y_test = train_test_split(
        dev_data[features],
        dev_data[target],
        test_size=0.2,
        random_state=42,
        stratify=dev_data[target]
    )

    print(f"\nData split:")
    print(f"Training samples: {len(X_train)}")
    print(f"Testing samples: {len(X_test)}")
    print(f"Positive class distribution in train: {(y_train==1).mean():.4f}")
    print(f"Positive class distribution in test: {(y_test==1).mean():.4f}")

    # Initialize and train model
    print("\nTraining model...")
    model = ImprovedCreditScorer()
    model.fit(X_train, y_train)

    # Evaluate on test set
    print("\nEvaluating on test set...")
    test_metrics = model.evaluate(X_test, y_test)

    # Create temporary directory for validation data
    temp_dir = 'temp_extracted'
    os.makedirs(temp_dir, exist_ok=True)

    try:
        # Extract and load validation data
        print("\nLoading validation data...")
        with zipfile.ZipFile('/content/validation_data_to_be_shared 3.zip', 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
            val_csv_csv_path = os.path.join(temp_dir, zip_ref.namelist()[0])

        val_data = pd.read_csv(val_csv_path)

        print("\nGenerating predictions for validation data...")
        # Retrain on full development data before making validation predictions
        print("\nRetraining model on full development data...")
        model = ImprovedCreditScorer()
        model.fit(dev_data[features], dev_data[target])

        # Generate predictions for validation data
        val_predictions = model.predict_proba(val_data[features])

        # Create submission file
        submission = pd.DataFrame({
            'account_number': val_data['account_number'],
            'predicted_probability': val_predictions
        })

        print("\nSaving predictions...")
        submission.to_csv('predictions_improved.csv', index=False)
        print("\nDone! Predictions saved to 'predictions_improved.csv'")

    finally:
        # Clean up temporary files
        import shutil
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

Loading development data...

Data split:
Training samples: 2376
Testing samples: 595
Positive class distribution in train: 0.1431
Positive class distribution in test: 0.1429

Training model...
Engineering features...
Scaling features...
Generating meta-features and CV scores...
Fold 1 AUC: 0.7471
Fold 2 AUC: 0.7431
Fold 3 AUC: 0.7970
Fold 4 AUC: 0.7700
Fold 5 AUC: 0.7814

Mean CV AUC: 0.7677 (+/- 0.0204)

Training base models on full data...

Evaluating on test set...

Model Performance Metrics:
Optimal Threshold: 0.8854
Accuracy: 0.8689
AUC Score: 0.8699
Average Precision: 0.5258

Confusion Matrix:
--------------------------------------------------
True Negatives: 476
False Positives: 34
False Negatives: 44
True Positives: 41

Additional Metrics:
--------------------------------------------------
Precision: 0.5467
Recall: 0.4824
F1 Score: 0.5125

Top 10 Important Features:
--------------------------------------------------

xgb1:
transaction_attribute_151    0.066835
bureau_12        